# Pruebo Bagging con distintos algoritmos #

Datos Filtrados

In [2]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np

from sklearn.metrics import SCORERS
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from mlens.ensemble import SuperLearner 

from itertools import combinations
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import MaxAbsScaler
from sklearn.kernel_approximation import RBFSampler

from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA



In [3]:
labels_f = pd.read_csv('labels_f_filtrado.csv')

In [4]:
datos = pd.read_csv('datos_filtrado.csv')

In [5]:
atributos = labels_f.columns.tolist()

In [6]:
atributos.remove('person')
atributos.remove('label')

In [7]:
y = labels_f['label']#.ravel()
X = labels_f.loc[:,atributos]

In [8]:
scaler = MaxAbsScaler().fit(X)
X = scaler.transform(X)

In [9]:
def aplicarSmote(X_train_p, y_train_p, X_test):
    smote = SMOTE(ratio='minority')
    X_smt_train, y_smt_train = smote.fit_sample(X_train_p, y_train_p)
    #X_smt_train = pd.DataFrame(X_smt_train)
    #X_smt_train.columns = X_test.columns
    return (X_smt_train, y_smt_train)

In [10]:
def aplicarRus(X_train_p, y_train_p,X_test_p):
    rus= RandomUnderSampler(return_indices=True)
    #id_rus son los índices
    X_rus_train, y_rus_train, id_rus = rus.fit_sample(X_train_p, y_train_p)
    #X_rus_train = pd.DataFrame(X_rus_train)
    #X_rus_train.columns = X_test_p.columns
    return(X_rus_train, y_rus_train)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

In [12]:
#X_train, y_train = aplicarRus(X_train,y_train)

In [13]:
X_rus_train, y_rus_train = aplicarRus(X_train,y_train,X_test)
X_smt_train, y_smt_train = aplicarSmote(X_train,y_train,X_test)

In [14]:
def aplicarModelo(modelo,X_train_p, X_test_p, y_train_p, y_test_p, early = False):
    if(early):
        modelo.fit(X_train_p, y_train_p, early_stopping_rounds = 5, eval_set=[(X_test_p, y_test_p)])
    else: 
        modelo.fit(X_train_p, y_train_p)
    
    preds = modelo.predict(X_test_p)
    preds_prob = modelo.predict_proba(X_test_p)[:,1]
    train_accuracy = accuracy_score(y_train_p, modelo.predict(X_train_p))
    test_accuracy = accuracy_score(y_test_p, preds)
    area_debajo_de_curva = roc_auc_score(y_test_p, preds_prob)
    matriz_de_confusion = confusion_matrix(y_test_p, preds)
    print('train acurracy: ')
    print(train_accuracy)
    print('test acurracy: ')
    print(test_accuracy)
    print('Matriz de confusión: ')
    print(matriz_de_confusion)
    print('Área bajo la curva: ')
    print(area_debajo_de_curva)
    print()

Pruebo con XGboost

In [15]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)




print('APLICO SMOTE')
aplicarModelo(xg_reg,X_smt_train, X_test, y_smt_train, y_test, True)
print('APLICO RUS')
aplicarModelo(xg_reg,X_rus_train, X_test, y_rus_train, y_test, True)

APLICO SMOTE
[0]	validation_0-error:0.371782
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.357364
[2]	validation_0-error:0.364058
[3]	validation_0-error:0.363543
[4]	validation_0-error:0.346035
[5]	validation_0-error:0.345005
train acurracy: 
0.8153563246111178
test acurracy: 
0.654994850669413
Matriz de confusión: 
[[1193  655]
 [  15   79]]
Área bajo la curva: 
0.826701667127199

APLICO RUS
[0]	validation_0-error:0.446447
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.554583
[2]	validation_0-error:0.528836
[3]	validation_0-error:0.515448
[4]	validation_0-error:0.522142
[5]	validation_0-error:0.511843
Stopping. Best iteration:
[0]	validation_0-error:0.446447

train acurracy: 
0.7533860045146726
test acurracy: 
0.5535530381050463
Matriz de confusión: 
[[988 860]
 [  7  87]]
Área bajo la curva: 
0.8238924196371005



Pruebo un Bagging de XGBOOST

In [16]:
bag_class =BaggingClassifier(xg_reg)
print('APLICO SMOTE')
aplicarModelo(bag_class,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(bag_class,X_rus_train, X_test, y_rus_train, y_test)

APLICO SMOTE
train acurracy: 
0.8030567948872543
test acurracy: 
0.6318228630278064
Matriz de confusión: 
[[1146  702]
 [  13   81]]
Área bajo la curva: 
0.823420373952289

APLICO RUS
train acurracy: 
0.739841986455982
test acurracy: 
0.4979402677651905
Matriz de confusión: 
[[876 972]
 [  3  91]]
Área bajo la curva: 
0.844734387952473



Pruebo con una red neuronal.

In [17]:
nn = MLPClassifier( early_stopping = True, random_state=123)


print('APLICO SMOTE')
aplicarModelo(nn,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(nn,X_rus_train, X_test, y_rus_train, y_test)

APLICO SMOTE
train acurracy: 
0.9481188954539973
test acurracy: 
0.872811534500515
Matriz de confusión: 
[[1659  189]
 [  58   36]]
Área bajo la curva: 
0.7551176660219213

APLICO RUS
train acurracy: 
0.7923250564334086
test acurracy: 
0.7842430484037075
Matriz de confusión: 
[[1455  393]
 [  26   68]]
Área bajo la curva: 
0.8325274016763379



Pruebo un Bagging de la red neuronal

In [18]:
bag_class =BaggingClassifier(xg_reg)
print('APLICO SMOTE')
aplicarModelo(bag_class,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(bag_class,X_rus_train, X_test, y_rus_train, y_test)

APLICO SMOTE
train acurracy: 
0.8048956951645967
test acurracy: 
0.635427394438723
Matriz de confusión: 
[[1154  694]
 [  14   80]]
Área bajo la curva: 
0.8235527770102238

APLICO RUS
train acurracy: 
0.7443566591422122
test acurracy: 
0.49948506694129763
Matriz de confusión: 
[[878 970]
 [  2  92]]
Área bajo la curva: 
0.8468787418255503




Pruebo con SVM con The Kernel trick

In [19]:
rbf_feature = RBFSampler(gamma=1,n_components= 100, random_state=123)
X_features = rbf_feature.fit_transform(X)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size= 0.1, random_state= 123)

In [21]:
X_rus_train, y_rus_train = aplicarRus(X_train,y_train,X_test)
X_smt_train, y_smt_train = aplicarSmote(X_train,y_train,X_test)

In [ ]:
clf = svm.SVC(kernel='linear', probability = True)

print('APLICO SMOTE')
aplicarModelo(clf,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(clf,X_rus_train, X_test, y_rus_train, y_test)

Pruebo SVM con The Kernel trick y bagging

In [ ]:
bag_class =BaggingClassifier(clf)

print('APLICO SMOTE')
aplicarModelo(bag_class,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(bag_class,X_rus_train, X_test, y_rus_train, y_test)

Reduzco dimensiones (PCA) y aplico KNN. Esto se debe a que KNN sufre la maldición de la dimensionalidad. 

In [ ]:
pca = PCA(n_components=10).fit(X)
X_reduce = pca.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reduce, y, test_size= 0.1, random_state= 123)

In [ ]:
X_rus_train, y_rus_train = aplicarRus(X_train,y_train,X_test)
X_smt_train, y_smt_train = aplicarSmote(X_train,y_train,X_test)



In [ ]:
knn = KNeighborsClassifier(n_neighbors= 80)

print('APLICO SMOTE')
aplicarModelo(knn,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(knn,X_rus_train, X_test, y_rus_train, y_test)

Aplico Bagging a KNN con PCA

In [ ]:
bag_class =BaggingClassifier(knn)

print('APLICO SMOTE')
aplicarModelo(bag_class,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(bag_class,X_rus_train, X_test, y_rus_train, y_test)

Aplico RandomForest

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, random_state= 123)

In [ ]:
X_rus_train, y_rus_train = aplicarRus(X_train,y_train,X_test)
X_smt_train, y_smt_train = aplicarSmote(X_train,y_train,X_test)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0)

print('APLICO SMOTE')
aplicarModelo(clf,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(clf,X_rus_train, X_test, y_rus_train, y_test)

Aplico bagging y random Forest

In [ ]:
bag_class =BaggingClassifier(clf)

print('APLICO SMOTE')
aplicarModelo(bag_class,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(bag_class,X_rus_train, X_test, y_rus_train, y_test)

xgboost y PCA

In [ ]:
pca = PCA(n_components=10).fit(X)
X_reduce = pca.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_reduce, y, test_size= 0.1, random_state= 123)

In [ ]:
X_rus_train, y_rus_train = aplicarRus(X_train,y_train,X_test)
X_smt_train, y_smt_train = aplicarSmote(X_train,y_train,X_test)

In [ ]:
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=15)


print('APLICO SMOTE')
aplicarModelo(xg_reg,X_smt_train, X_test, y_smt_train, y_test)
print('APLICO RUS')
aplicarModelo(xg_reg,X_rus_train, X_test, y_rus_train, y_test)

DE ACA EN ADELANTE NO MIRAR

In [228]:
bag_class =BaggingClassifier(xg_reg,max_samples = 1000)
aplicarModelo(bag_class,X_train, X_test, y_train, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/bagging.py:621: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.840293453724605
test acurracy: 
0.6848609680741503
Matriz de confusión: 
[[1271  577]
 [  35   59]]
Área bajo la curva: 
0.7283952749378282


Hago un ensamble de Xgboost usando super learner (este lo sacao pq me dio mal)

In [65]:
#xg_reg = xgb.XGBClassifier(learning_rate =0.05,n_estimators=10,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
#xg_reg_2 = xgb.XGBClassifier(learning_rate =0.01,n_estimators=10,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
#xg_reg_3 = xgb.XGBClassifier(learning_rate =0.1,n_estimators=100,max_depth=5,min_child_weight=1,gamma=0,
                           subsample=0.8,colsample_bytree=0.8,objective= 'binary:logistic',nthread=4,
                            seed=27)
#gaussiano =GaussianNB()

#clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

#knn = KNeighborsClassifier(n_neighbors=4)

#ensemble = SuperLearner(scorer = accuracy_score, random_state = 123, folds = 10)
#ensemble.add(gaussiano)
#ensemble.add(clf)
#ensemble.add(knn)
#ensemble.add(xg_reg_2)
#ensemble.add(xg_reg_3)
#ensemble.add_meta(xg_reg)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


train acurracy: 
0.9536973443223443
test acurracy: 
0.952626158599382
Matriz de confusión: 
[[1847    1]
 [  91    3]]
Área bajo la curva: 
0.5156868840379479
